## Selección de hiper-parámetros

La mayoria de estimadores tienen hiper-parámetros que hay que ajustar para que el rendimiento sea bueno. Por ejemplo ya hemos visto el parámetro $k$ de vecinos próximos:

<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/knn_boundary_varying_k.png width=500>

En la mayoría de estimadores estos hiper-parámetros representan la "flexibilidad" del modelo:

   * Modelos muy flexibles son capaces de memorizar el conjunto de entrenamiento, pero tendrán un mal rendimiento en el conjunto de test (sobreajuste)
   * Modelos poco flexibles no serán capaces de aprender el patrón de los datos, y tendrán un mal rendimiento en general (infraajuste)
   
Este equilibrio entre sobreajuste e infraajuste se suele representar de manera teórica con gráficos como este:

<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/overfitting_underfitting_cartoon_full.png width=500>

Es importante destacar que no es un gráfico realizado con datos reales, sino que es un esquema de lo que se suele observar en la práctica. Hasta el momento dividíamos nuestros datos en dos conjuntos:

   * entrenamiento
   * test
   
<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/train_test_split_new.png width=500>

Pero el conjunto de test **no** se puede usar para comparar el mismo modelo con distintos hiper-parámetros y elegir el que mejor resultado tenga. El conjunto de test **solo** se usa para dar una estimación del rendimiento final. 

### Conjunto de validación

Por tanto, para ajustar hiper-parámetros vamos a dividir los datos en tres conjuntos:

  * entramiento
  * test
  * validación
  
<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/train_test_validation_split.png width=500>

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

ds = load_breast_cancer()

In [6]:
ds.data.shape

(569, 30)

In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(ds.data, ds.target, stratify=ds.target, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, stratify=y_train_val, random_state=0)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(319, 30)
(143, 30)
(107, 30)


In [9]:
from sklearn.neighbors import KNeighborsClassifier

val_scores = {}
for k in (1, 3, 5, 10, 15):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    val_scores[k] = knn.score(X_val, y_val)

In [10]:
import pandas as pd
val_series = pd.Series(val_scores)
val_series

1     0.925234
3     0.915888
5     0.915888
10    0.943925
15    0.925234
dtype: float64

In [11]:
k_best = val_series.idxmax()

knn = KNeighborsClassifier(n_neighbors=k_best)
knn.fit(X_train_val, y_train_val)

print(f'k óptimo: {k_best}')
print(f'acierto entrenamiento: {knn.score(X_train_val, y_train_val):.3f}')
print(f'acierto test: {knn.score(X_test, y_test):.3f}')

k óptimo: 10
acierto entrenamiento: 0.946
acierto test: 0.916


### Validación cruzada

Consiste en partir los datos de entrenamiento en varios subconjuntos e ir rotando el conjunto de validación:

<img src=../../img/cross_validation_new.png width=500>

Ventajas con respecto a tener un único conjunto de validación:

   * Más estable
   * Se usan todos los datos de entrenamiento para buscar los parámetros óptimos
   
Desventajas:

   * Más lento, hay que ajustar tantos modelos como subconjuntos

In [12]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

cv_scores = {}
for k in (1, 3, 5, 10, 15):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_val, y_train_val)
    cv_scores[k] = np.mean(cross_val_score(knn, X_train_val, y_train_val, cv=10))

In [13]:
import pandas as pd
cv = pd.Series(cv_scores)
cv

1     0.924806
3     0.938926
5     0.943632
10    0.941307
15    0.934275
dtype: float64

#### Búsqueda en rejilla + validación cruzada

La estrategia de buscar los hiper-parámetros óptimos calculando el error de validación cruzada para cada valor de los parámetros en una rejilla es bastante habitual.

scikit-learn implementa esta estrategia en la clase `GridSearchCV`, que simplifica el proceso anterior:

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11],
    'p': [1, 2]
}

cv = GridSearchCV(
    KNeighborsClassifier(), 
    param_grid=param_grid, 
    cv=10
)
cv.fit(X_train_val, y_train_val)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11], 'p': [1, 2]})

In [17]:
cv.score(X_test, y_test)

0.916083916083916

In [18]:
import pandas as pd

pd.DataFrame(cv.cv_results_).sort_values(by='mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
4,0.000160,0.000006,0.001349,0.000113,5,1,"{'n_neighbors': 5, 'p': 1}",0.930233,0.976744,0.953488,0.953488,0.976744,0.953488,0.928571,1.00000,0.928571,0.952381,0.955371,0.022346,1
2,0.000259,0.000154,0.001959,0.000652,3,1,"{'n_neighbors': 3, 'p': 1}",0.953488,0.953488,0.953488,0.953488,0.976744,0.906977,0.928571,1.00000,0.952381,0.952381,0.953101,0.023429,2
6,0.000177,0.000021,0.001555,0.000198,7,1,"{'n_neighbors': 7, 'p': 1}",0.930233,0.953488,0.953488,0.953488,0.953488,0.906977,0.976190,1.00000,0.928571,0.928571,0.948450,0.025240,3
8,0.000162,0.000006,0.001330,0.000049,9,1,"{'n_neighbors': 9, 'p': 1}",0.930233,0.953488,0.953488,0.953488,0.953488,0.906977,0.976190,1.00000,0.928571,0.928571,0.948450,0.025240,3
10,0.000168,0.000017,0.001405,0.000151,11,1,"{'n_neighbors': 11, 'p': 1}",0.930233,0.953488,0.953488,0.953488,0.953488,0.906977,0.952381,0.97619,0.928571,0.928571,0.943688,0.018714,5
5,0.000180,0.000020,0.001688,0.000411,5,2,"{'n_neighbors': 5, 'p': 2}",0.930233,0.953488,0.953488,0.953488,0.953488,0.930233,0.928571,0.97619,0.928571,0.928571,0.943632,0.015774,6
11,0.000163,0.000012,0.001502,0.000165,11,2,"{'n_neighbors': 11, 'p': 2}",0.930233,0.953488,0.930233,0.953488,0.953488,0.906977,0.928571,1.00000,0.928571,0.928571,0.941362,0.024084,7
7,0.000166,0.000012,0.001479,0.000165,7,2,"{'n_neighbors': 7, 'p': 2}",0.930233,0.953488,0.953488,0.953488,0.953488,0.906977,0.928571,0.97619,0.928571,0.928571,0.941307,0.018969,8
9,0.000163,0.000008,0.001492,0.000141,9,2,"{'n_neighbors': 9, 'p': 2}",0.930233,0.953488,0.930233,0.953488,0.953488,0.930233,0.928571,0.97619,0.928571,0.928571,0.941307,0.015864,8
3,0.000160,0.000005,0.001506,0.000227,3,2,"{'n_neighbors': 3, 'p': 2}",0.930233,0.930233,0.953488,0.953488,0.953488,0.930233,0.928571,0.97619,0.904762,0.928571,0.938926,0.019061,10


In [19]:
print(f'k óptimo: {cv.best_params_}')
print(f'mejor cv score: {cv.best_score_:.3f}')
print(f'acierto test: {cv.score(X_test, y_test):.3f}')

k óptimo: {'n_neighbors': 5, 'p': 1}
mejor cv score: 0.955
acierto test: 0.916


### Otras estrategias de validación cruzada

La estrategia anterior se conoce con el nombre de validación cruzada de $k$ hojas o $k$-fold cross-validation. Como hemos visto, consiste en crear $k$ subconjuntos aleatorios de forma aleatoria con probabilidad uniforme.

También existen otras estrategias:

   1. `StratifiedKFold`, generar los subconjuntos de forma que se mantengan la proporción de las clases (estratificados). Esto es especialmente importante si las clases **no están balanceadas**. En `GridSearchCV` y `cross_val_score` la validación cruzada está **estratificada** por defecto si el estimador es de clasificación
   
   2. `LeaveOneOut`: validación cruzada de $k$ hojas con $k=1$
   
   3. `(Stratified)ShuffleSplit`: muestrea el conjunto de test con reemplazamiento
   
   4. `Repeated(Stratified)KFold`: repite la validación cruzada múltiples veces

#### Validación cruzada en series temporales

Para las series temporales no tiene sentido escoger el conjunto de test de manera aleatoria, ya que estamos "prediciendo" usando datos del futuro
 
Conjunto de test aleatorio | Conjunto de test estructurado
---|---
<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/time_series2.png style="width:100%"> | <img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/time_series3.png style="width:100%">

La clase `TimeSeriesSplit` realiza una validación cruzada pero manteniendo la estructura temporal:

<img src=https://amueller.github.io/ml-workshop-2-of-4/slides/images/time_series_cv.png width=500>

De esta forma, si por ejemplo tenemos datos de 1 año:

   * *Primera partición*: Enero-Octubre entrenamiento, Noviembre-Diciembre test
   * *Segunda partición*: Enero-Agosto entrenamiento, Septiembre-Octubre test
   * *Tercera partición*: Enero-Junio entrenamiento, Julio-Agosto test
   * etc
   
Es importante destacar que el conjunto de test siempre tiene el mismo tamaño (2 meses en este caso), pero el conjunto de entrenamiento tiene tamaño variable.

In [12]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

param_grid = {'n_neighbors': np.arange(1, 20, 2)}
grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=cv)
grid.fit(X_train_val, y_train_val)

print(f'k óptimo: {grid.best_params_}')
print(f'mejor cv score: {grid.best_score_:.3f}')
print(f'acierto test: {grid.score(X_test, y_test):.3f}')

k óptimo: {'n_neighbors': 11}
mejor cv score: 0.936
acierto test: 0.916


### Ejercicios

Con el conjunto de datos de titanic:

   1. Preparar los datos de la misma forma que en el notebook `03-missing.ipynb`, imputando los valores que faltan de `age` con la media
   
   2. Ajustar un modelo `LogisticRegression` y buscar el valor óptimo del parámetro `C`

In [13]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
 
titanic = sns.load_dataset('titanic')
 
X = titanic.drop(columns=['deck', 'embarked', 'alive', 'survived', 'class', 'who'])
y = titanic['survived']
 
# ojo, estamos completando tanto train como test, pero como faltan pocos valores en este caso no hay problema
X.loc[X['embark_town'].isna(), 'embark_town'] = X['embark_town'].mode()
X_num = pd.get_dummies(X, drop_first=True)
 
X_train, X_test, y_train, y_test = train_test_split(X_num, y, stratify=y, random_state=0)
 
clf = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                      ('classifier', LogisticRegression(solver='liblinear'))])
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8071748878923767

In [14]:
param_grid = {
    'classifier__C': [0.1, 1.0, 2, 10, 100],
}

grid= GridSearchCV(clf, param_grid, cv=10)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)

0.8071748878923767

In [15]:
grid.best_params_

{'classifier__C': 10}